# VGG classifier training and fine tunning comparison

In this notebook I want to compare the accuracy of a VGG model build from scratch and trained on the target dataset (bike category recognition) with a pre trained model (ImageNet) and fine tunned model.

I'll be using discriminative fine-tunning in this notebook 

In [54]:
!pip3 install -r requirements.txt

Python(94417) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [55]:
from dotenv import load_dotenv
from utils.load_dataset import load_dataset
import os

load_dotenv()
dataset_url = os.getenv("DATASET_REPO_URL")

load_dataset(dataset_url, extract_dir="dataset")



ℹ️  Detected classes: ['cargo', 'road', 'fold', 'mtb', 'hybrid']
🔄 Processing: cargo...
🔄 Processing: road...
🔄 Processing: fold...
🔄 Processing: mtb...
🔄 Processing: hybrid...

✅ Done! Dataset organized at: dataset_split


In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import os

In [57]:
SEED = 43

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Architecture definition

In [58]:
class VGG(nn.Module):
    def __init__(self, features, output_dim):
        super().__init__()

        self.features = features

        self.avgpool = nn.AdaptiveAvgPool2d(7)

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

### VGG configuration arrays

In [59]:
vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

vgg13_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512,
                512, 'M']

vgg16_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512,
                'M', 512, 512, 512, 'M']

vgg19_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512,
                512, 512, 'M', 512, 512, 512, 512, 'M']

In [60]:
def get_vgg_layers(config, batch_norm):

    layers = []
    in_channels = 3

    for c in config:
        assert c == 'M' or isinstance(c, int)
        if c == 'M':
            layers += [nn.MaxPool2d(kernel_size=2)]
        else:
            conv2d = nn.Conv2d(in_channels, c, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = c

    return nn.Sequential(*layers)

### ResNet-18 Architecture

In [61]:
import torchvision.models as models

def get_resnet18(num_classes, pretrained=False):
    """
    Returns a ResNet-18 model with modified final layer for num_classes.
    Skip connections differentiate it from VGG's plain architecture.
    """
    if pretrained:
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    else:
        model = models.resnet18(weights=None)
    
    # Replace final fully connected layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    
    return model

# Wrapper to match VGG interface (returns outputs, hidden)
class ResNetWrapper(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super().__init__()
        self.resnet = get_resnet18(num_classes, pretrained)
        # Store feature extractor (everything except fc)
        self.features = nn.Sequential(*list(self.resnet.children())[:-1])
        self.fc = self.resnet.fc
    
    def forward(self, x):
        # Extract features
        h = self.features(x)
        h = h.view(h.size(0), -1)
        # Classify
        out = self.fc(h)
        return out, h

# Test instantiation
resnet18_model = ResNetWrapper(num_classes=5, pretrained=False)
print(f"ResNet-18 created with {sum(p.numel() for p in resnet18_model.parameters()):,} parameters")

ResNet-18 created with 11,179,077 parameters


In [62]:
vgg11_layers = get_vgg_layers(vgg11_config, batch_norm=True)

vgg16_layers = get_vgg_layers(vgg16_config, batch_norm=True)

print(vgg11_layers)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU(inplace=True)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [63]:
from torch.utils.data import DataLoader

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# =============================================================================
# 4 AUGMENTATION VARIANTS (A1-A4)
# =============================================================================

# A1: Baseline - No augmentation (only resize + normalize)
aug_A1_baseline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# A2: Geometric - Spatial transformations
aug_A2_geometric = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# A3: Color - Color/intensity transformations
aug_A3_color = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# A4: Heavy - All augmentations combined
aug_A4_heavy = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Validation/Test transform (always deterministic)
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Dictionary for easy access
AUGMENTATION_VARIANTS = {
    'A1_baseline': aug_A1_baseline,
    'A2_geometric': aug_A2_geometric,
    'A3_color': aug_A3_color,
    'A4_heavy': aug_A4_heavy,
}

print("Augmentation variants defined:")
for name in AUGMENTATION_VARIANTS:
    print(f"  - {name}")

Augmentation variants defined:
  - A1_baseline
  - A2_geometric
  - A3_color
  - A4_heavy


## Data Scenarios (E1-E4)

In [64]:
# =============================================================================
# DATA SCENARIOS (E1-E4)
# =============================================================================

DATA_SCENARIOS = {
    'E1_full': 'dataset_split/full',        # All data (real + synthetic)
    'E2_synthetic': 'dataset_split/synthetic',  # Only synthetic
    'E3_real': 'dataset_split/real',        # Only real
    'E4_balanced': 'dataset_split/real',    # Balanced (handled separately)
}

def data_loader(data_dir, batch_size, set_type='train', shuffle=True, transform=None):
    """
    Creates a DataLoader for the specified dataset split.
    """
    if transform is None:
        transform = val_transforms
    set_path = os.path.join(data_dir, set_type)
    dataset = datasets.ImageFolder(set_path, transform=transform)
    loader = DataLoader(dataset, batch_size, shuffle=shuffle)
    size = len(dataset)
    return loader, size, dataset.classes

def get_balanced_subset(data_dir, set_type='train', target_size_per_class=None):
    """
    Creates a balanced subset by limiting each class to target_size_per_class.
    If target_size_per_class is None, uses the minimum class size.
    """
    set_path = os.path.join(data_dir, set_type)
    dataset = datasets.ImageFolder(set_path, transform=None)
    
    # Group samples by class
    class_indices = {}
    for idx, (_, label) in enumerate(dataset.samples):
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(idx)
    
    # Find minimum class size if not specified
    if target_size_per_class is None:
        target_size_per_class = min(len(indices) for indices in class_indices.values())
    
    # Sample equally from each class
    balanced_indices = []
    for label, indices in class_indices.items():
        if len(indices) >= target_size_per_class:
            balanced_indices.extend(random.sample(indices, target_size_per_class))
        else:
            balanced_indices.extend(indices)  # Use all if fewer
    
    return balanced_indices, target_size_per_class

def create_data_loaders(scenario, batch_size, aug_variant='A1_baseline'):
    """
    Creates train, val, test loaders for a given scenario and augmentation.
    
    Returns: train_loader, val_loader, test_loader, class_names, sizes_dict
    """
    train_transform = AUGMENTATION_VARIANTS.get(aug_variant, aug_A1_baseline)
    
    if scenario == 'E4_balanced':
        # Special handling for balanced dataset
        data_dir = DATA_SCENARIOS['E3_real']  # Base on real data
        
        # Get balanced indices
        balanced_indices, samples_per_class = get_balanced_subset(data_dir, 'train')
        
        # Create dataset with transform
        train_path = os.path.join(data_dir, 'train')
        full_dataset = datasets.ImageFolder(train_path, transform=train_transform)
        train_dataset = torch.utils.data.Subset(full_dataset, balanced_indices)
        train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
        train_size = len(train_dataset)
        class_names = full_dataset.classes
        
    else:
        data_dir = DATA_SCENARIOS[scenario]
        train_loader, train_size, class_names = data_loader(
            data_dir, batch_size, 'train', shuffle=True, transform=train_transform
        )
    
    # Val and test always use real data with deterministic transforms
    val_loader, val_size, _ = data_loader(
        DATA_SCENARIOS['E3_real'], batch_size, 'val', shuffle=False, transform=val_transforms
    )
    test_loader, test_size, _ = data_loader(
        DATA_SCENARIOS['E3_real'], batch_size, 'test', shuffle=False, transform=val_transforms
    )
    
    sizes = {'train': train_size, 'val': val_size, 'test': test_size}
    
    return train_loader, val_loader, test_loader, class_names, sizes

# Test data scenarios
print("Data Scenarios:")
for scenario, path in DATA_SCENARIOS.items():
    if os.path.exists(os.path.join(path, 'train')):
        _, size, classes = data_loader(path, 1, 'train')
        print(f"  {scenario}: {size} train samples, classes={classes}")
    else:
        print(f"  {scenario}: path not found")

Data Scenarios:
  E1_full: 97 train samples, classes=['cargo', 'fold', 'hybrid', 'mtb', 'road']
  E2_synthetic: 50 train samples, classes=['cargo', 'fold', 'hybrid', 'mtb', 'road']
  E3_real: 48 train samples, classes=['cargo', 'fold', 'hybrid', 'mtb', 'road']
  E4_balanced: 48 train samples, classes=['cargo', 'fold', 'hybrid', 'mtb', 'road']


## Training & Evaluation Functions

In [65]:
num_classes = 5
num_epochs = 50
batch_size = 32
learning_rate = 1e-4
weight_decay = 1e-4

model = VGG(vgg11_layers, num_classes)

model16 = VGG(vgg16_layers, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = weight_decay)
optimizer16 = torch.optim.Adam(model16.parameters(), lr=learning_rate, weight_decay = weight_decay)  

### Training loop

In [66]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs, device=None, verbose=True):
    """
    Trains a model and returns training history.
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = model.to(device)
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_acc = 0.0
    best_model_state = None

    for epoch in range(num_epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs, _ = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / max(total, 1)
        train_acc = 100.0 * correct / max(total, 1)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs, _ = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * labels.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / max(val_total, 1)
        val_acc = 100.0 * val_correct / max(val_total, 1)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = copy.deepcopy(model.state_dict())

        if verbose:
            print(f"Epoch [{epoch+1}/{num_epochs}] "
                  f"train_loss={train_loss:.4f} train_acc={train_acc:.2f}% | "
                  f"val_loss={val_loss:.4f} val_acc={val_acc:.2f}%")

    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, history, best_val_acc

In [67]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

def evaluate_model(model, test_loader, class_names, device=None):
    """
    Evaluates model on test set and returns metrics + confusion matrix.
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs, _ = model(images)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Accuracy
    accuracy = 100.0 * (all_preds == all_labels).sum() / len(all_labels)
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    # Precision, Recall, F1
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro', zero_division=0
    )
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision * 100,
        'recall': recall * 100,
        'f1_score': f1 * 100,
        'confusion_matrix': cm,
        'predictions': all_preds,
        'labels': all_labels,
    }
    
    return metrics

def plot_confusion_matrix(cm, class_names, title='Confusion Matrix'):
    """
    Plots a confusion matrix.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=ax, cmap='Blues', values_format='d')
    ax.set_title(title)
    plt.tight_layout()
    return fig

def plot_training_history(history, title='Training History'):
    """
    Plots training and validation loss/accuracy curves.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Train Loss')
    axes[0].plot(history['val_loss'], label='Val Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title(f'{title} - Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # Accuracy
    axes[1].plot(history['train_acc'], label='Train Acc')
    axes[1].plot(history['val_acc'], label='Val Acc')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title(f'{title} - Accuracy')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    return fig

print("Training & Evaluation functions defined.")

Training & Evaluation functions defined.


## Experiment Runner (32 Experiments)

In [69]:
!pip3 install pandas

Python(96336) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


  Using cached pandas-3.0.0-cp311-cp311-macosx_10_9_x86_64.whl (10.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.0
    Uninstalling numpy-1.24.0:
      Successfully uninstalled numpy-1.24.0

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [70]:
import pandas as pd

# =============================================================================
# EXPERIMENT CONFIGURATION
# =============================================================================

ARCHITECTURES = {
    'VGG16': lambda num_classes: VGG(get_vgg_layers(vgg16_config, batch_norm=True), num_classes),
    'ResNet18': lambda num_classes: ResNetWrapper(num_classes, pretrained=False),
}

# Experiment hyperparameters
EXPERIMENT_CONFIG = {
    'num_classes': 5,
    'num_epochs': 30,
    'batch_size': 16,
    'learning_rate': 1e-4,
    'weight_decay': 1e-4,
}

def run_single_experiment(arch_name, scenario, aug_variant, config=EXPERIMENT_CONFIG, verbose=True):
    """
    Runs a single experiment and returns results.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n{'='*60}")
    print(f"Experiment: {arch_name} | {scenario} | {aug_variant}")
    print(f"Device: {device}")
    print(f"{'='*60}")
    
    # Create data loaders
    train_loader, val_loader, test_loader, class_names, sizes = create_data_loaders(
        scenario, config['batch_size'], aug_variant
    )
    print(f"Data sizes: train={sizes['train']}, val={sizes['val']}, test={sizes['test']}")
    print(f"Classes: {class_names}")
    
    # Create model
    model = ARCHITECTURES[arch_name](config['num_classes'])
    
    # Setup training
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), 
        lr=config['learning_rate'], 
        weight_decay=config['weight_decay']
    )
    
    # Train
    model, history, best_val_acc = train_model(
        model, train_loader, val_loader, criterion, optimizer,
        config['num_epochs'], device, verbose=verbose
    )
    
    # Evaluate on test set
    metrics = evaluate_model(model, test_loader, class_names, device)
    
    print(f"\n--- Test Results ---")
    print(f"Accuracy: {metrics['accuracy']:.2f}%")
    print(f"Precision: {metrics['precision']:.2f}%")
    print(f"Recall: {metrics['recall']:.2f}%")
    print(f"F1 Score: {metrics['f1_score']:.2f}%")
    
    # Package results
    result = {
        'architecture': arch_name,
        'scenario': scenario,
        'augmentation': aug_variant,
        'train_size': sizes['train'],
        'val_size': sizes['val'],
        'test_size': sizes['test'],
        'best_val_acc': best_val_acc,
        'test_accuracy': metrics['accuracy'],
        'test_precision': metrics['precision'],
        'test_recall': metrics['recall'],
        'test_f1': metrics['f1_score'],
        'model': model,
        'history': history,
        'confusion_matrix': metrics['confusion_matrix'],
        'class_names': class_names,
    }
    
    return result

def run_all_experiments(architectures=None, scenarios=None, augmentations=None, config=EXPERIMENT_CONFIG):
    """
    Runs all 32 experiments (or subset) and returns results DataFrame.
    """
    if architectures is None:
        architectures = list(ARCHITECTURES.keys())
    if scenarios is None:
        scenarios = list(DATA_SCENARIOS.keys())
    if augmentations is None:
        augmentations = list(AUGMENTATION_VARIANTS.keys())
    
    total = len(architectures) * len(scenarios) * len(augmentations)
    print(f"Running {total} experiments...")
    print(f"Architectures: {architectures}")
    print(f"Scenarios: {scenarios}")
    print(f"Augmentations: {augmentations}")
    
    all_results = []
    
    for arch in architectures:
        for scenario in scenarios:
            for aug in augmentations:
                try:
                    result = run_single_experiment(arch, scenario, aug, config, verbose=False)
                    all_results.append(result)
                    
                    # Save intermediate results
                    model_name = f"{arch}_{scenario}_{aug}"
                    torch.save(result['model'].state_dict(), f"models/{model_name}.pth")
                    
                except Exception as e:
                    print(f"ERROR in {arch}/{scenario}/{aug}: {e}")
                    all_results.append({
                        'architecture': arch,
                        'scenario': scenario,
                        'augmentation': aug,
                        'error': str(e),
                    })
    
    # Create summary DataFrame
    summary_data = []
    for r in all_results:
        if 'error' not in r:
            summary_data.append({
                'Architecture': r['architecture'],
                'Scenario': r['scenario'],
                'Augmentation': r['augmentation'],
                'Train Size': r['train_size'],
                'Best Val Acc (%)': r['best_val_acc'],
                'Test Acc (%)': r['test_accuracy'],
                'Test Precision (%)': r['test_precision'],
                'Test Recall (%)': r['test_recall'],
                'Test F1 (%)': r['test_f1'],
            })
    
    results_df = pd.DataFrame(summary_data)
    return results_df, all_results

print("Experiment runner configured.")
print(f"Architectures: {list(ARCHITECTURES.keys())}")
print(f"Scenarios: {list(DATA_SCENARIOS.keys())}")
print(f"Augmentations: {list(AUGMENTATION_VARIANTS.keys())}")
print(f"Total experiments: {len(ARCHITECTURES) * len(DATA_SCENARIOS) * len(AUGMENTATION_VARIANTS)}")

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python -m pip install -ve . --no-build-isolation -Ceditable-verbose=true' to build the C extensions first.

## Run Single Experiment (Test)

In [ ]:
# Test single experiment (VGG16, real data, baseline augmentation)
test_result = run_single_experiment('VGG16', 'E3_real', 'A1_baseline', 
                                     config={**EXPERIMENT_CONFIG, 'num_epochs': 5})

# Plot confusion matrix
plot_confusion_matrix(test_result['confusion_matrix'], test_result['class_names'], 
                      title='VGG16 - E3_real - A1_baseline')
plt.show()

# Plot training history
plot_training_history(test_result['history'], title='VGG16 - E3_real - A1_baseline')
plt.show()

## Run All 32 Experiments

In [ ]:
# Run all experiments (this will take a while!)
# Uncomment to run:

# results_df, all_results = run_all_experiments()
# results_df.to_csv('experiment_results.csv', index=False)
# print(results_df.to_string())

# Or run a subset:
results_df, all_results = run_all_experiments(
    architectures=['VGG16', 'ResNet18'],
    scenarios=['E3_real'],  # Start with real data only
    augmentations=['A1_baseline', 'A4_heavy'],
    config={**EXPERIMENT_CONFIG, 'num_epochs': 10}
)
print("\n" + "="*80)
print("RESULTS SUMMARY")
print("="*80)
print(results_df.to_string())

## Results Visualization & Comparison

In [ ]:
def plot_comparison_charts(results_df):
    """
    Creates comparison visualizations for experiment results.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # 1. Architecture comparison
    if len(results_df['Architecture'].unique()) > 1:
        arch_comparison = results_df.groupby('Architecture')['Test Acc (%)'].mean()
        arch_comparison.plot(kind='bar', ax=axes[0, 0], color=['steelblue', 'coral'])
        axes[0, 0].set_title('Average Test Accuracy by Architecture')
        axes[0, 0].set_ylabel('Accuracy (%)')
        axes[0, 0].set_xticklabels(axes[0, 0].get_xticklabels(), rotation=0)
    
    # 2. Data scenario comparison
    if len(results_df['Scenario'].unique()) > 1:
        scenario_comparison = results_df.groupby('Scenario')['Test Acc (%)'].mean()
        scenario_comparison.plot(kind='bar', ax=axes[0, 1], color='teal')
        axes[0, 1].set_title('Average Test Accuracy by Data Scenario')
        axes[0, 1].set_ylabel('Accuracy (%)')
        axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=45)
    
    # 3. Augmentation comparison
    if len(results_df['Augmentation'].unique()) > 1:
        aug_comparison = results_df.groupby('Augmentation')['Test Acc (%)'].mean()
        aug_comparison.plot(kind='bar', ax=axes[1, 0], color='purple')
        axes[1, 0].set_title('Average Test Accuracy by Augmentation')
        axes[1, 0].set_ylabel('Accuracy (%)')
        axes[1, 0].set_xticklabels(axes[1, 0].get_xticklabels(), rotation=45)
    
    # 4. Heatmap: Architecture x Scenario
    if len(results_df['Architecture'].unique()) > 1 and len(results_df['Scenario'].unique()) > 1:
        pivot = results_df.pivot_table(
            values='Test Acc (%)', 
            index='Architecture', 
            columns='Scenario', 
            aggfunc='mean'
        )
        im = axes[1, 1].imshow(pivot.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
        axes[1, 1].set_xticks(range(len(pivot.columns)))
        axes[1, 1].set_xticklabels(pivot.columns, rotation=45)
        axes[1, 1].set_yticks(range(len(pivot.index)))
        axes[1, 1].set_yticklabels(pivot.index)
        axes[1, 1].set_title('Test Accuracy: Architecture x Scenario')
        plt.colorbar(im, ax=axes[1, 1])
        
        # Annotate cells
        for i in range(len(pivot.index)):
            for j in range(len(pivot.columns)):
                axes[1, 1].text(j, i, f'{pivot.values[i, j]:.1f}%', 
                               ha='center', va='center', fontsize=10)
    
    plt.tight_layout()
    return fig

def plot_all_confusion_matrices(all_results, max_cols=4):
    """
    Plots confusion matrices for all experiments in a grid.
    """
    valid_results = [r for r in all_results if 'error' not in r]
    n = len(valid_results)
    if n == 0:
        print("No valid results to plot.")
        return
    
    n_cols = min(max_cols, n)
    n_rows = (n + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 4*n_rows))
    if n == 1:
        axes = np.array([[axes]])
    elif n_rows == 1:
        axes = axes.reshape(1, -1)
    elif n_cols == 1:
        axes = axes.reshape(-1, 1)
    
    for idx, result in enumerate(valid_results):
        row, col = idx // n_cols, idx % n_cols
        ax = axes[row, col]
        
        cm = result['confusion_matrix']
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=result['class_names'])
        disp.plot(ax=ax, cmap='Blues', values_format='d', colorbar=False)
        ax.set_title(f"{result['architecture']}\n{result['scenario']}\n{result['augmentation']}", fontsize=9)
    
    # Hide empty subplots
    for idx in range(n, n_rows * n_cols):
        row, col = idx // n_cols, idx % n_cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    return fig

# Plot comparisons if results exist
if 'results_df' in dir() and len(results_df) > 0:
    plot_comparison_charts(results_df)
    plt.show()
    
    plot_all_confusion_matrices(all_results)
    plt.show()

## Save Final Results

In [ ]:
# Save results to CSV
if 'results_df' in dir() and len(results_df) > 0:
    results_df.to_csv('experiment_results.csv', index=False)
    print("Results saved to experiment_results.csv")
    print("\nFinal Results Table:")
    print(results_df.to_string())

# Save best model
if 'all_results' in dir() and len(all_results) > 0:
    valid_results = [r for r in all_results if 'error' not in r and 'test_accuracy' in r]
    if valid_results:
        best = max(valid_results, key=lambda x: x['test_accuracy'])
        best_name = f"best_{best['architecture']}_{best['scenario']}_{best['augmentation']}.pth"
        torch.save(best['model'].state_dict(), f"models/{best_name}")
        print(f"\nBest model saved: {best_name}")
        print(f"  Test Accuracy: {best['test_accuracy']:.2f}%")